### Visualization of crack propagation in space and time

In [1]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import DBSCAN

Specify dilation factor

In [2]:
df = 6

In [3]:
node = np.genfromtxt("newimg1-MLd"+str(df)+"-node.txt",delimiter = ',')
link = np.genfromtxt("newimg1-MLd"+str(df)+"-link.txt",delimiter = ',')

In [4]:
numElem = link.shape[0]

Specify number of frames

In [5]:
numFrames = 34

Import status as numElem x 2 x numFrames array

In [6]:
statuses = np.genfromtxt("statuses/newimg1-MLd"+str(df)+"-mises_status.txt",delimiter = ',',max_rows = numElem*numFrames)

In [7]:
statuses = np.transpose(np.reshape(statuses.T,(2, numElem, numFrames),order = 'F'),(1,0,2))

In [8]:
row_ind = np.argsort(statuses[:,0,:],0)

In [9]:
for i in range(0,statuses.shape[2]):
    sslice = statuses[:,:,i]
    sslice = sslice[row_ind[:,i]]
    statuses[:,:,i] = sslice

Calculate link centroids

In [10]:
centroids = np.zeros((link.shape[0],2))

In [11]:
for i in range(0,link.shape[0]):
    centroids[i,0] = (node[int(link[i,1]-1),1] + node[int(link[i,2]-1),1])/2
    centroids[i,1] = (node[int(link[i,1]-1),2] + node[int(link[i,2]-1),2])/2

Plot cracked points at time t

In [12]:
t = 29

In [13]:
crackpts = np.vstack((centroids[np.nonzero(statuses[:,1,t]==0)[0],0],centroids[np.nonzero(statuses[:,1,t]==0)[0],1])).T

In [14]:
plt.figure()
plt.scatter(centroids[:,0],centroids[:,1],color='blue',zorder=0)
plt.scatter(crackpts[:,0],crackpts[:,1],color='yellow',edgecolors='k',zorder=1)
plt.show()

FigureCanvasNbAgg()

Find clusters with DBSCAN algorithm

In [15]:
db = DBSCAN(eps=3, min_samples=10).fit(crackpts)

In [16]:
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

In [17]:
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print(n_clusters_)

2


In [18]:
unique_labels = set(labels)
colors = [plt.cm.Paired(each)
          for each in np.linspace(0, 1, len(unique_labels))]
plt.figure()
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = crackpts[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=7)

    xy = crackpts[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=3)

plt.show()

FigureCanvasNbAgg()

In [19]:
t_start = 27
fig = plt.figure()
ax = fig.add_subplot(111,projection = '3d')
for t in range(t_start,t_start+5):
    crackpts = np.vstack((centroids[np.nonzero(statuses[:,1,t]==0)[0],0],centroids[np.nonzero(statuses[:,1,t]==0)[0],1])).T
    ax.scatter(crackpts[:,0],crackpts[:,1],t*np.ones_like(crackpts[:,0]))

ax.set_zlabel('time', rotation=90)
plt.show()
    

FigureCanvasNbAgg()